In [ ]:
from google.colab import drive 
drive.mount('/content/drive',)

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/lfw3.zip"  "lfw3.zip"
!unzip "lfw3.zip" 

Streaming output truncated to the last 5000 lines.
  inflating: lfw3/lfw-deepfunneled_masked/Jennifer_Lopez/Jennifer_Lopez_0005_surgical.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Candie_Kung/Candie_Kung_0004.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Igor_Ivanov/Igor_Ivanov_0019.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Benjamin_Netanyahu/Benjamin_Netanyahu_0001.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Jean-Marc_de_La_Sabliere/Jean-Marc_de_La_Sabliere_0001.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Jenny_Romero/Jenny_Romero_0001_surgical.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Laura_Bush/Laura_Bush_0018.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Jim_Ahern/Jim_Ahern_0001.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/John_Kerr/John_Kerr_0001.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Candie_Kung/Candie_Kung_0003.jpg  
  inflating: lfw3/lfw-deepfunneled_masked/Jennifer_Lopez/Jennifer_Lopez_0004_surgical.jpg  
  inflating: lfw3/lfw-deepfunne

In [ ]:
from tensorflow.keras.regularizers import l2
import pathlib
import tensorflow 
from tensorflow import keras
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense,Dropout,BatchNormalization
import tensorflow.keras 
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.utils as utils
from tensorflow.keras.optimizers import Adam as adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad 
from tensorflow.keras.callbacks import  EarlyStopping ,ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout, Input

from tensorflow.keras.applications import VGG16

from tensorflow.keras.applications.resnet50 import preprocess_input
from matplotlib import pyplot
from PIL import Image
from numpy import asarray

import glob
import cv2
from tensorflow.keras import layers

In [ ]:
def get_paths():
  classes = []
  for file in sorted(glob.iglob('/content/lfw3/lfw-deepfunneled_masked/*/')):
    classes.append(file)
  for i,d in enumerate(classes):
        paths=d+'*.jpg'
        class_=[]     
        for file in sorted(glob.iglob(paths)):
            class_.append(file)
        classes[i]=class_ 
  return classes

In [ ]:
from google.colab.patches import cv2_imshow
def get_all_images(classes):
  allImages=[]
  Labels = []
  count = 0
  for k,d in enumerate(classes[:]):
    for i,sample in enumerate(d):
      org_img = cv2.imread(sample)
      #org_img = org_img.astype('float32')
      org_img = cv2.resize(org_img, (224, 224))
      # org_img=cv2.cvtColor(org_img,cv2.COLOR_BGR2RGB)
      # np.append(allImages, org_img)
      allImages.append(org_img)
      Labels.append(count)
    count = count + 1   
  return np.array(allImages),np.array(Labels)

In [ ]:
classes=get_paths()
classes

[['/content/lfw3/lfw-deepfunneled_masked/AJ_Cook/AJ_Cook_0001.jpg',
  '/content/lfw3/lfw-deepfunneled_masked/AJ_Cook/AJ_Cook_0001_surgical.jpg'],
 ['/content/lfw3/lfw-deepfunneled_masked/AJ_Lamas/AJ_Lamas_0001.jpg',
  '/content/lfw3/lfw-deepfunneled_masked/AJ_Lamas/AJ_Lamas_0001_surgical.jpg'],
 ['/content/lfw3/lfw-deepfunneled_masked/Aaron_Eckhart/Aaron_Eckhart_0001.jpg',
  '/content/lfw3/lfw-deepfunneled_masked/Aaron_Eckhart/Aaron_Eckhart_0001_surgical.jpg'],
 ['/content/lfw3/lfw-deepfunneled_masked/Aaron_Guiel/Aaron_Guiel_0001.jpg',
  '/content/lfw3/lfw-deepfunneled_masked/Aaron_Guiel/Aaron_Guiel_0001_surgical.jpg'],
 ['/content/lfw3/lfw-deepfunneled_masked/Aaron_Patterson/Aaron_Patterson_0001.jpg',
  '/content/lfw3/lfw-deepfunneled_masked/Aaron_Patterson/Aaron_Patterson_0001_surgical.jpg'],
 ['/content/lfw3/lfw-deepfunneled_masked/Aaron_Peirsol/Aaron_Peirsol_0001.jpg',
  '/content/lfw3/lfw-deepfunneled_masked/Aaron_Peirsol/Aaron_Peirsol_0001_surgical.jpg',
  '/content/lfw3/lfw-deep

In [ ]:
X,Y=get_all_images(classes)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.05,shuffle=True, random_state=777)
x_train1, x_val1, y_train1, y_val1 = train_test_split(x_train, y_train, test_size=0.2,shuffle=True, random_state=463)
x_train2, x_val2, y_train2, y_val2 = train_test_split(x_train, y_train, test_size=0.2,shuffle=True, random_state=749)

# Transfer_learning

In [ ]:
from keras.models import load_model
# load the model
def Transfer_FacenetModel_withNormlization():
  
  facenetmodel = tf.keras.models.load_model('/content/drive/MyDrive/FaceNet/facenet_keras.h5')
  facenetmodel.load_weights('/content/drive/MyDrive/FaceNet/facenet_keras_weights.h5')
  for layer in facenetmodel.layers[:-50]:
    layer.trainable = False
  inputs = layers.Input(shape=(256,256,3))
    # Augment data.
  augmented = data_augmentation(inputs)
  # This is 'bootstrapping' a new top_model onto the pretrained layers.
  top_model = facenetmodel(augmented)
  top_model = Dropout(0.5)(top_model)
  top_model = BatchNormalization()(top_model)
  top_model = Flatten(name="flatten")(top_model)
  output_layer = Dense(5750, activation='softmax')(top_model)

  # Group the convolutional base and new fully-connected layers into a Model object.
  model = Model(inputs=inputs, outputs=output_layer)

  return model

In [ ]:
data_augmentation = keras.Sequential(
    [
        
        layers.Resizing(160, 160),
    ],
    name="data_augmentation",
)

In [ ]:
Facenet1=Transfer_FacenetModel_withNormlization()
Facenet1.load_weights('/content/drive/MyDrive/FaceNet/MyEn3facenet.h5')
Facenet2=Transfer_FacenetModel_withNormlization()
Facenet2.load_weights('/content/drive/MyDrive/FaceNet/MyEn4facenet.h5')

# Transformer

In [ ]:
#Ensamble Learning Training

learning_rate = 0.001

weight_decay = 0.0001

batch_size = 32

num_epochs = 300

image_size = 72  # We'll resize input images to this size

patch_size = 6  # Size of the patches to be extract from the input images

num_patches = (image_size // patch_size) ** 2

projection_dim = 64

num_heads = 8

transformer_units = [

    projection_dim * 2,

    projection_dim,

]  # Size of the transformer layers

transformer_layers = 10

mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
data_augmentation = keras.Sequential(

    [

        layers.Rescaling(1./255),

        layers.Resizing(image_size, image_size),

        layers.RandomFlip("horizontal")

        #,layers.RandomRotation(0.3)

    ],

    name="data_augmentation",

)

# Compute the mean and the variance of the training data for normalization.

#data_augmentation.layers[0].adapt(x_train)

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():

    inputs = layers.Input(shape=(224,224,3))

    # Augment data.

    augmented = data_augmentation(inputs)

    # Create patches.

    patches = Patches(patch_size)(augmented)

    # Encode patches.

    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)



    # Create multiple layers of the Transformer block.

    for _ in range(transformer_layers):

        # Layer normalization 1.

        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

        # Create a multi-head attention layer.

        attention_output = layers.MultiHeadAttention(

            num_heads=num_heads, key_dim=projection_dim, dropout=0.3

        )(x1, x1)

        # Skip connection 1.

        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 2.

        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)

        # MLP.

        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.3)

        # Skip connection 2.

        encoded_patches = layers.Add()([x3, x2])



    # Create a [batch_size, projection_dim] tensor.

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

    representation = layers.Flatten()(representation)

    representation = layers.Dropout(0.6)(representation)

    # Add MLP.

    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.6)

    # Classify outputs.

    logits = layers.Dense(5750)(features)

    # Create the Keras model.

    model = keras.Model(inputs=inputs, outputs=logits)

    return model

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
transformerClassifier1 = create_vit_classifier()
transformerClassifier2 = create_vit_classifier()

In [ ]:
transformerClassifier1.load_weights('/content/drive/MyDrive/FaceNet/FirstTransformer3Ensamble1.h5')
transformerClassifier2.load_weights('/content/drive/MyDrive/FaceNet/FirstTransformer3Ensamble2.h5')

# Ensemble Part

In [ ]:
pred_prob1=Facenet1.predict(x_test)
preds_classes1 = np.argmax(pred_prob1, axis=-1)
preds_classes1

array([1872,  708, 4079, ..., 1893, 1277, 4868])

In [ ]:
pred_prob2=Facenet2.predict(x_test)
preds_classes2 = np.argmax(pred_prob2, axis=-1)
preds_classes2

array([1872,  708, 2089, ..., 1893, 1277, 4868])

In [ ]:
layer12 = tf.keras.layers.Softmax()

pred_prob3 = layer12(transformerClassifier1.predict(x_test)).numpy()
pred_prob4 = layer12(transformerClassifier2.predict(x_test)).numpy()

In [ ]:
preds_classes4 = np.argmax(pred_prob4, axis=-1)
preds_classes4

array([1872,  514, 2089, ..., 1893, 1277, 4868])

In [ ]:
from sklearn.metrics import accuracy_score
pred_sum=pred_prob1+pred_prob2+pred_prob3+pred_prob4
preds_classes_sum = np.argmax(pred_sum, axis=-1)
accuracy_score(y_test, preds_classes_sum)

0.9201520912547528